In [1]:
import sys
sys.path.append('..')

In [2]:
import asyncio
import aiohttp

from misc.asynctools import agnostic

In [3]:
async def get_one(*args, session=None, **kwargs):
    if session is None:
        async with aiohttp.ClientSession() as session:
            return await get_one(*args, session=session, **kwargs)
    async with session.get(*args, **kwargs) as response:
        try:
            response.body = await response.read()
            response.sucess = True
        except Exception as exception:
            response.exception = exception
            response.sucess = False
        return response

In [4]:
r = asyncio.run(get_one('http://python.org'))
r.body.decode(r.charset)[:1000]

'<!doctype html>\n<!--[if lt IE 7]>   <html class="no-js ie6 lt-ie7 lt-ie8 lt-ie9">   <![endif]-->\n<!--[if IE 7]>      <html class="no-js ie7 lt-ie8 lt-ie9">          <![endif]-->\n<!--[if IE 8]>      <html class="no-js ie8 lt-ie9">                 <![endif]-->\n<!--[if gt IE 8]><!--><html class="no-js" lang="en" dir="ltr">  <!--<![endif]-->\n\n<head>\n    <meta charset="utf-8">\n    <meta http-equiv="X-UA-Compatible" content="IE=edge">\n\n    <link rel="prefetch" href="//ajax.googleapis.com/ajax/libs/jquery/1.8.2/jquery.min.js">\n\n    <meta name="application-name" content="Python.org">\n    <meta name="msapplication-tooltip" content="The official home of the Python Programming Language">\n    <meta name="apple-mobile-web-app-title" content="Python.org">\n    <meta name="apple-mobile-web-app-capable" content="yes">\n    <meta name="apple-mobile-web-app-status-bar-style" content="black">\n\n    <meta name="viewport" content="width=device-width, initial-scale=1.0">\n    <meta name="Han

In [5]:
async def get_all(urls, *args, session=None, **kwargs):
    if session is None:
        _len = len(urls)
        connector = aiohttp.TCPConnector(limit=_len, limit_per_host=_len)
        async with aiohttp.ClientSession(connector=connector) as session:
            return await get_all(urls, *args, session=session, **kwargs)
    return await asyncio.gather(*(
        get_one(url, *args, session=session, **kwargs)
        for url in urls
    ))

In [6]:
rs = asyncio.run(get_all([
    'http://python.org',
    'http://python.org',
    'http://python.org',
    'http://nodejs.org',
    'http://nodejs.org',
    'http://nodejs.org',
    'http://php.net',
    'http://php.net',
    'http://php.net',
]))

for r in rs:
    print(r.sucess, r.status)

True 200
True 200
True 200
True 200
True 200
True 200
True 200
True 200
True 200
